<a href="https://colab.research.google.com/github/OguzhanKececiler/100MtThreadKosu-/blob/main/100MKo%C5%9Fu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

In [ ]:
%load_ext nvcc_plugin

In [ ]:
%%writefile Kosucu.h
#pragma once
class Kosucu{
  public:
    Kosucu(int _id);
    int id;
    float mesafe;
    float hiz;
    bool bitti;
};

In [ ]:
%%writefile Kosucu.cpp
#include "Kosucu.h"
Kosucu::Kosucu(int _id){
    id=_id;
    mesafe=0;
    hiz=0;
    bitti=false;
};

In [ ]:
%%writefile main.cu
#include <curand_kernel.h>
#include <thrust/sort.h>
#include <thread>
#include "Kosucu.h"

using namespace std;
using namespace std::chrono;

__host__ void yaz(Kosucu *h_kosucular, int adet){
    printf("Siralama:");
  for (int i = 0; i < adet; i++)
    printf("%d\t\t",i+1);
  printf("\n");
  printf("Mesafe:\t ");
  for (int i = 0; i < adet; i++)
    printf("%f\t",h_kosucular[i].mesafe);
  printf("\n");
  printf("Id:\t ");
  for (int i = 0; i < adet; i++)
    printf("%d\t\t",h_kosucular[i].id);
  printf("\n\n\n");
}
__global__ void setup_kernel(curandState *state){
  int idx = threadIdx.x;
  curand_init(clock64(), idx, 0, &state[idx]);
}
__global__ void generate_kernel(curandState *my_curandstate, Kosucu *kosucular){
  int idx = threadIdx.x;
  if(kosucular[idx].bitti)
    return;
  float myrandf = curand_uniform(my_curandstate+idx);
  myrandf *= 10000;
  myrandf = fmodf(myrandf,4);
  kosucular[idx].hiz = myrandf;
}
__host__ void rndHizAtama(int adet, Kosucu *h_kosucular){
  curandState *d_state;
  cudaMalloc(&d_state, adet * sizeof(curandState));

  Kosucu *d_kosucular;
  cudaMalloc(&d_kosucular, adet * sizeof(Kosucu));
  
  cudaMemcpy(d_kosucular, h_kosucular, adet * sizeof(Kosucu), cudaMemcpyHostToDevice);

  setup_kernel<<<1,adet>>>(d_state);
  generate_kernel<<<1,adet>>>(d_state, d_kosucular);
  
  cudaMemcpy(h_kosucular, d_kosucular, adet * sizeof(Kosucu), cudaMemcpyDeviceToHost);
}
__global__ void keySet(float *d_key, Kosucu *d_kosucular){
  int idx = threadIdx.x;
  d_key[idx]=d_kosucular[idx].mesafe;
}
__host__ void sirala(int adet, Kosucu *h_kosucular){
  Kosucu *d_kosucular;
  cudaMalloc(&d_kosucular, adet * sizeof(Kosucu));
  cudaMemcpy(d_kosucular, h_kosucular, adet * sizeof(Kosucu), cudaMemcpyHostToDevice);
  
  float *d_key, *h_key;
  h_key = (float *)malloc(adet * sizeof(float));
  cudaMalloc(&d_key, adet * sizeof(float));
  //cudaMemset(d_key, 0,adet * sizeof(float));

  for (int i = 0; i < adet; i++)
    h_key[i]=i+1;
  //cudaMemcpy(d_key, h_key, adet * sizeof(Kosucu), cudaMemcpyHostToDevice);
  
  keySet<<<1,adet>>>(d_key, d_kosucular);
  cudaMemcpy(h_key, d_key, adet * sizeof(float), cudaMemcpyDeviceToHost);
  
  thrust::sort_by_key(thrust::host, h_key, h_key + adet, h_kosucular, thrust::greater<float>());  
}
__global__ void kos(Kosucu *kosucular, bool *flag, int pist){
  int idx = threadIdx.x;
  kosucular[idx].mesafe = kosucular[idx].mesafe + kosucular[idx].hiz;
  if(kosucular[idx].mesafe>pist){    
    flag[0]=true;
    kosucular[idx].hiz=5; 
    kosucular[idx].bitti=true; 
  }
}
__host__ void kostur(int adet, Kosucu *h_kosucular, int pist, int maxHiz, int minHiz, bool *h_flag, bool *finish){//

  Kosucu *d_kosucular;
  cudaMalloc(&d_kosucular, adet * sizeof(Kosucu));
  cudaMemcpy(d_kosucular, h_kosucular, adet * sizeof(Kosucu), cudaMemcpyHostToDevice);

  bool *d_flag;
  cudaMalloc(&d_flag, 2 * sizeof(bool));
  cudaMemcpy(d_flag, h_flag, 2 * sizeof(bool), cudaMemcpyHostToDevice);
    
  rndHizAtama(adet, d_kosucular);
  kos<<<1,adet>>>(d_kosucular, d_flag, pist);

  cudaMemcpy(h_kosucular, d_kosucular, adet * sizeof(Kosucu), cudaMemcpyDeviceToHost);
  cudaMemcpy(h_flag, d_flag, 2 * sizeof(bool), cudaMemcpyDeviceToHost);

  if(h_flag[0])
  sirala(adet, h_kosucular);
  
  if(h_flag[0]&&!finish[0]){
    printf("\nBirinci Finişi Geçti...\n");
    yaz(h_kosucular, adet);
    finish[0]=true;
  }
  h_flag[1]=h_kosucular[adet-1].bitti;

  if(h_flag[1]){
    printf("\nYarışma Bitti...\n");
    yaz(h_kosucular, adet);
  }
}

int main(){
  int adet = 100; //koşucu adedi
  int pist = 100; //koşu mesafesi
  int minHiz=1;
  int maxHiz=5;
  float turSuresi=1;//sn bazında

  Kosucu *kosucular;
  kosucular = (Kosucu *)malloc(adet * sizeof(Kosucu));
  for (int i = 0; i < adet; i++){
      Kosucu *k=new Kosucu(i+1);
      kosucular[i]=*k;
  }

  bool *flag, *finish;
  flag =   (bool *)malloc(2 * sizeof(bool));
  finish = (bool *)malloc(2 * sizeof(bool));
  flag[0]=false;
  finish[0]=false;
  flag[1]=false;
  finish[1]=false;
  int turAdedi=0; 
  const clock_t start = clock();

  const clock_t time = clock();
  clock_t next=time+CLOCKS_PER_SEC*turSuresi;
  while(!flag[1]) { 
    const clock_t time = clock();
    if(time>=next){
      kostur(adet, kosucular, pist, maxHiz, minHiz, flag, finish);
      turAdedi++;
      next+=CLOCKS_PER_SEC*turSuresi;
    } 
  }
 
} 

In [ ]:
%%script bash
nvcc -arch=sm_37 main.cu Kosucu.cpp -o Race 

In [ ]:
!./Race